In [3]:
#Install required libraries
!pip install fastapi uvicorn psycopg2-binary sqlalchemy python-dotenv groq


In [7]:
#Load secrets
import os
from google.colab import userdata
os.environ["SUPABASE_DB_HOST"] = userdata.get("SUPABASE_DB_HOST")
os.environ["SUPABASE_DB_PORT"] = userdata.get("SUPABASE_DB_PORT")
os.environ["SUPABASE_DB_NAME"] = userdata.get("SUPABASE_DB_NAME")
os.environ["SUPABASE_DB_USER"] = userdata.get("SUPABASE_DB_USER")
os.environ["SUPABASE_DB_PASSWORD"] = userdata.get("SUPABASE_DB_PASSWORD")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [8]:
#connection
from dotenv import load_dotenv
import os
import psycopg2

# Load .env
load_dotenv()

try:
    # Connect to Supabase
    conn = psycopg2.connect(
        host=os.getenv("SUPABASE_DB_HOST"),
        port=int(os.getenv("SUPABASE_DB_PORT")),
        database=os.getenv("SUPABASE_DB_NAME"),
        user=os.getenv("SUPABASE_DB_USER"),
        password=os.getenv("SUPABASE_DB_PASSWORD"),
        sslmode="require",
        connect_timeout=10,
    )

    print("Connected successfully!")

    # Optional test query
    cur = conn.cursor()
    cur.execute("SELECT current_user;")
    print("User:", cur.fetchone())

except psycopg2.OperationalError as e:
    print(" Database connection failed")
    print(e)

except Exception as e:
    print(" Unexpected error")
    print(e)

finally:
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()
        print(" Connection closed")



Connected successfully!
User: ('postgres',)
 Connection closed


In [9]:
#SQL SAFETY GUARD
def is_safe_sql(sql: str) -> bool:
    forbidden = ["insert", "update", "delete", "drop", "truncate", "alter"]
    sql_lower = sql.lower().strip()
    return sql_lower.startswith("select") and not any(
        f" {word} " in f" {sql_lower} " for word in forbidden
    )




In [10]:
#row limit safety:

def enforce_limit(sql: str, limit: int = 1000) -> str:
    if "limit" not in sql.lower():
        return sql.rstrip(";") + f" LIMIT {limit};"
    return sql


In [11]:
# SQL Executor

from psycopg2.extras import RealDictCursor

def execute_sql(conn, sql: str):
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute(sql)
        return cur.fetchall()

In [12]:
#NL2SQL System Prompt

# system_prompt
SYSTEM_PROMPT = """
You are an expert PostgreSQL NL2SQL agent.

Your task is to convert a user’s natural language business question
(English or Hinglish) into a correct, safe PostgreSQL SELECT query.

If the question is in Hindi/Hinglish, internally translate it to English
before generating SQL.

Return ONLY the SQL query.
Do NOT add explanations, comments, or markdown.

--------------------------------------------------
DATABASE SCHEMA
--------------------------------------------------

table_first:
- date, firm, firm_product, voucher_type, voucher_no,
  gstin_uin, quantity, rate, value, brand,
  sub_category, category, client_type

table_second:
- date, firm, voucher_no, quantity, value,
  gstin_uin, total_tax_amount, gross_total

--------------------------------------------------
BUSINESS SEMANTIC LAYER
--------------------------------------------------

- Sales / business / turnover / revenue → gross_total (table_second)
- Net sales → value
- GST / tax → total_tax_amount (ALWAYS use table_second for these queries, including any grouping columns like 'firm' if present in table_second)
- Product / brand / category / quantity → table_first

--------------------------------------------------
JOIN RULES
--------------------------------------------------

- Do NOT use JOIN unless strictly required
- JOIN ONLY on voucher_no
- If a query can be answered using table_second alone, JOIN is INCORRECT
- Avoid joins that cause double counting

--------------------------------------------------
AGGREGATION RULES
--------------------------------------------------

- Always use COALESCE(SUM(column), 0)
- Do not aggregate unless explicitly asked

--------------------------------------------------
GROUPING (“WISE”) RULES
--------------------------------------------------

If question contains "firm wise", "brand wise", "category wise",
then:
- SELECT that column
- GROUP BY that column

--------------------------------------------------
DATE RULES
--------------------------------------------------

- Use EXTRACT(YEAR FROM date) and EXTRACT(MONTH FROM date) for the relevant table's date column.
- Month filters MUST include year

Current:
- "is month" → CURRENT_DATE month + year
- "is saal" → CURRENT_DATE year

Past:
- "pichle saal" → current_year - 1
- "pichle N saal" → BETWEEN current_year-N AND current_year-1

Future:
- Only filter existing data
- Do NOT predict

--------------------------------------------------
LIMIT & ORDER
--------------------------------------------------

- Add LIMIT only if user asks
- Top / highest → ORDER BY aggregate DESC
- Lowest → ORDER BY aggregate ASC

--------------------------------------------------
OUTPUT
--------------------------------------------------

Return ONLY valid PostgreSQL SELECT SQL.
"""

In [13]:
#NL2SQL Agent (Groq)
from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def nl_to_sql(question: str) -> str:
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question},
        ],
        temperature=0,
    )
    return response.choices[0].message.content.strip()


In [14]:
def get_connection():
    load_dotenv()
    return psycopg2.connect(
        host=os.getenv("SUPABASE_DB_HOST"),
        port=int(os.getenv("SUPABASE_DB_PORT")),
        database=os.getenv("SUPABASE_DB_NAME"),
        user=os.getenv("SUPABASE_DB_USER"),
        password=os.getenv("SUPABASE_DB_PASSWORD"),
        sslmode="require",
        connect_timeout=10,
    )

In [15]:
# FastAPI Endpoint
from pydantic import BaseModel
#Define request model

class QueryRequest(BaseModel):
    question: str
#Update the endpoint
from fastapi import FastAPI, HTTPException
app = FastAPI()
@app.post("/query")
def query_db(payload: QueryRequest):
    question = payload.question

    sql = nl_to_sql(question)

    # Safety checks
    if not is_safe_sql(sql):
        raise HTTPException(status_code=400, detail="Unsafe SQL blocked")

    sql = enforce_limit(sql)

    conn = get_connection()
    try:
        data = execute_sql(conn, sql)
    except Exception as e:
        raise HTTPException(
            status_code=400,
            detail=f"SQL execution failed: {str(e)}"
        )
    finally:
        conn.close()

    return {
        "question": question,
        "sql": sql,
        "data": data,
    }

In [16]:
#check fast api
import nest_asyncio
import uvicorn
import threading
import time
import os
import signal

nest_asyncio.apply()

def run_server():
    # Check if the port is in use and kill the process if it is
    try:
        # Find process using port 8001
        output = os.popen(f"lsof -t -i:8001").read()
        if output:
            pid = int(output.strip().split()[0])
            os.kill(pid, signal.SIGTERM)
            print(f"Killed existing process on port 8001 (PID: {pid})")
            time.sleep(1) # Give a moment for the port to release
    except Exception as e:
        print(f"Error checking/killing process on port 8001: {e}")

    uvicorn.run(app, host="0.0.0.0", port=8001, log_level="warning")

# Ensure the thread is not already running to avoid multiple servers
if 'thread' in locals() and thread.is_alive():
    print("Server thread already running.")
else:
    thread = threading.Thread(target=run_server,daemon=True)
    thread.start()

time.sleep(2)
print("🚀 API running on http://localhost:8001")

🚀 API running on http://localhost:8001


##Testing the Endpoint


In [17]:
import requests

BASE_URL = "http://localhost:8001"

response = requests.post(
    f"{BASE_URL}/query",
    json={"question": "What was firmwise sales in December 2025"}
)

print("Status:", response.status_code)
print(response.json())


Status: 200
{'question': 'What was firmwise sales in December 2025', 'sql': 'SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, firm, COALESCE(SUM(value), 0) AS sales \nFROM table_first \nWHERE EXTRACT(MONTH FROM date) = 12 AND EXTRACT(YEAR FROM date) = 2025 \nGROUP BY firm, EXTRACT(YEAR FROM date), EXTRACT(MONTH FROM date) LIMIT 1000;', 'data': [{'year': 2025, 'month': 12, 'firm': 'A TO Z SECUTECH PRIVATE LIMITED', 'sales': 1347652.0}, {'year': 2025, 'month': 12, 'firm': 'ABN CONNECT', 'sales': 44160.93}, {'year': 2025, 'month': 12, 'firm': 'ACS DIGITAL INDIA (AKASH)', 'sales': 104877.06}, {'year': 2025, 'month': 12, 'firm': 'ADINATH SECURITY SOLUTIONS', 'sales': 10000.0}, {'year': 2025, 'month': 12, 'firm': 'ADVANCE COMMUNICATION SERVICE', 'sales': 720.34}, {'year': 2025, 'month': 12, 'firm': 'ADVANCE SECURITY & IT', 'sales': 285593.41}, {'year': 2025, 'month': 12, 'firm': 'AJEEM', 'sales': 1059.32}, {'year': 2025, 'month': 12, 'firm': 'ALCOFAB ALUMINIUM WORK 

In [18]:
response = requests.post(
    f"{BASE_URL}/query",
    json={"question": "Category wise sales for December 2025"}
)

print("Status:", response.status_code)
print(response.json())

Status: 200
{'question': 'Category wise sales for December 2025', 'sql': 'SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, category, COALESCE(SUM(value), 0) AS sales \nFROM table_first \nWHERE EXTRACT(MONTH FROM date) = 12 AND EXTRACT(YEAR FROM date) = 2025 \nGROUP BY category, EXTRACT(YEAR FROM date), EXTRACT(MONTH FROM date) LIMIT 1000;', 'data': [{'year': 2025, 'month': 12, 'category': 'Accessories', 'sales': 15810336.45}, {'year': 2025, 'month': 12, 'category': 'Cameras', 'sales': 8772320.47}]}


In [19]:
response = requests.post(
    f"{BASE_URL}/query",
    json={"question": "Give sales for November 2025"}
)

print("Status:", response.status_code)
print(response.json())

Status: 200
{'question': 'Give sales for November 2025', 'sql': 'SELECT COALESCE(SUM(value), 0) FROM table_first WHERE EXTRACT(YEAR FROM date) = EXTRACT(YEAR FROM CURRENT_DATE) AND EXTRACT(MONTH FROM date) = EXTRACT(MONTH FROM CURRENT_DATE) LIMIT 1000;', 'data': [{'coalesce': 0.0}]}


In [20]:
response = requests.post(
    f"{BASE_URL}/query",
    json={"question": "Total sales for the year 2024?"}
)

print("Status:", response.status_code)
print(response.json())


Status: 200
{'question': 'Total sales for the year 2024?', 'sql': 'SELECT COALESCE(SUM(value), 0) FROM table_first WHERE EXTRACT(YEAR FROM date) = 2024 LIMIT 1000;', 'data': [{'coalesce': 0.0}]}
